<a href="https://colab.research.google.com/github/eldrey/topicos_contemporaneos/blob/main/Part%202%20-%20LLMs/05%20-%20HuggingFace%20Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# Utilizando Modelos Pré-treinados com HuggingFace

Hugging Face é uma plataforma que oferece uma ampla variedade de modelos pré-treinados para tarefas de processamento de linguagem natural (NLP). Usar esses modelos permite que você aplique técnicas avançadas de NLP sem precisar treinar um modelo do zero.

## Tokenizadores

Tokenização é o processo de converter texto em tokens, que são as unidades básicas que os modelos de NLP processam. A Hugging Face fornece tokenizadores para diferentes modelos, que geram tokens compatíveis com o modelo que será utilizado.

Nesta seção, aprenderemos como carregar e utilizar tokenizadores com Hugging Face.

In [ ]:
from transformers import GPT2Tokenizer, BertTokenizer

# Carregando tokenizadores para GPT-2 e BERT
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Exemplo de tokenização
input_text = "Artificial intelligence is the future."
gpt2_tokens = gpt2_tokenizer.tokenize(input_text)
bert_tokens = bert_tokenizer.tokenize(input_text)

print(f"Tokens GPT-2: {gpt2_tokens}")
print(f"Tokens BERT: {bert_tokens}")

In [ ]:
gpt2_ids = gpt2_tokenizer.convert_tokens_to_ids(gpt2_tokens)
bert_ids = bert_tokenizer.convert_tokens_to_ids(bert_tokens)

print(f"IDs GPT-2: {gpt2_ids}")
print(f"IDs BERT: {bert_ids}")

In [ ]:
gpt2_input = gpt2_tokenizer(input_text, return_tensors="pt")
bert_input = bert_tokenizer(input_text, return_tensors="pt")

print(f"Input GPT-2: {gpt2_input}")
print(f"Input BERT: {bert_input}")

## Modelos

Modelos são as redes neurais que processam os tokens e geram saídas como texto ou embeddings. A Hugging Face disponibiliza uma variedade de modelos, como GPT-2 para geração de texto e BERT para tarefas como busca semântica.

Aqui, aprenderemos como carregar e usar esses modelos para diferentes tarefas.

### Carregando Modelos Pré-Treinados

Para começar, vamos carregar modelos pré-treinados, como o GPT-2.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Carregando o tokenizador GPT-2
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Carregando o modelo GPT-2
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=gpt2_tokenizer.eos_token_id)

## Geração de Texto

Geração de texto consiste em fornecer uma sequência inicial e permitir que o modelo continue gerando texto a partir dessa entrada. Vamos ver como fazer isso com o GPT-2.

A geração de texto é uma das aplicações mais comuns de modelos como o GPT-2, que podem ser usados para completar frases, criar histórias ou mesmo gerar código.

In [ ]:
# Texto de entrada
input_text = "In a world where AI"

# Tokenizando a entrada
input_ids = gpt2_tokenizer(input_text, return_tensors="pt")

print(f"Input IDs: {input_ids}")

In [ ]:
# Gerando o texto
output = gpt2_model.generate(**input_ids, max_length=50)

print(output.shape)
print(f"Output: {output}")

In [ ]:
# Decodificando o texto gerado
generated_text = gpt2_tokenizer.decode(output[0])
print(generated_text)

In [ ]:
# Função para completar texto
def complete_text(input_text, model=gpt2_model, tokenizer=gpt2_tokenizer, max_length=50):
    input_ids = tokenizer(input_text, return_tensors="pt")
    output = model.generate(**input_ids, max_length=max_length)
    generated_text = tokenizer.decode(output[0])
    return generated_text

In [ ]:
prediction = complete_text("Brazil is")
print(prediction)

In [ ]:
def generate_next_tokens(input_text, n_tokens=1, model=gpt2_model, tokenizer=gpt2_tokenizer):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_new_tokens=n_tokens)
    generated_tokens = output[0][len(input_ids[0]):]  # Extrai apenas os novos tokens gerados
    predicted_tokens = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return predicted_tokens.strip()

input_text = "Brazil is a country. Apple is a fruit. Python is a"

next_token = generate_next_tokens(input_text)
print(next_token)

## Embeddings de Texto

Embeddings de texto são representações vetoriais de palavras ou frases que capturam o significado semântico. Esses embeddings podem ser utilizados em várias tarefas de NLP, como classificação de texto, busca semântica e agrupamento.

Nesta seção, utilizaremos o BERT para gerar embeddings de texto e visualizar a similaridade entre diferentes frases. Também introduziremos conceitos como similaridade por cosseno.

In [ ]:
from transformers import BertTokenizer, BertModel

# Carregando o tokenizador BERT
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Carregando o modelo BERT
bert_model = BertModel.from_pretrained("bert-base-uncased")

## Capturando Embeddings

Predição de embeddings consiste em fornecer um texto e utilizar um modelo encoder para prever a representação vetorial deste texto. Vamos ver como fazer isso com o BERT.

In [ ]:
# Texto de entrada
text = "Artificial intelligence is the future."
input_ids = bert_tokenizer.encode(text, return_tensors="pt")

# Obtendo a representação do texto
with torch.no_grad():
    output = bert_model(input_ids)


pooled_output = output.last_hidden_state.mean(dim=1)

print(output.last_hidden_state.shape)
print(pooled_output.shape)

In [ ]:
# Função para gerar embeddings usando BERT
def get_embedding(text):
    input_ids = bert_tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(input_ids)
    return outputs.last_hidden_state.mean(dim=1)

### Medidas de Similaridade

A similaridade por cosseno é uma métrica comum utilizada para medir a similaridade entre dois vetores no espaço de embeddings. Ela calcula o cosseno do ângulo entre dois vetores, onde um valor de 1 indica vetores idênticos e um valor de 0 indica vetores ortogonais (sem similaridade).

A fórmula da similaridade por cosseno é:

$
\text{similaridade}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$

Vamos calcular a similaridade entre diferentes frases usando embeddings gerados por BERT.

In [ ]:
# Exemplo de frases
query = "Artificial intelligence is transforming industries."
doc1 = "AI is changing the way we work."
doc2 = "Brazil is a country in South America."

# Gerando embeddings
query_embedding = get_embedding(query)
doc1_embedding = get_embedding(doc1)
doc2_embedding = get_embedding(doc2)

In [ ]:
# Calculando similaridade por cosseno
cos = torch.nn.CosineSimilarity(dim=1)
similarity_doc1 = cos(query_embedding, doc1_embedding)
similarity_doc2 = cos(query_embedding, doc2_embedding)

print(f"Similaridade com doc1: {similarity_doc1.item():.4f}")
print(f"Similaridade com doc2: {similarity_doc2.item():.4f}")

## Exercícios

### Exercício 1
Utilizando GPT-2, crie uma função que preveja o sujeito em uma frase. Por exemplo: Em "John went to the store", o sujeito é John.

### Exercício 2
Em um sistema avançado de busca por livros, você deverá implementar uma função que faça uma busca semântica e retorne os 5 livros mais apropriados de acordo com a consulta do usuário.

In [ ]:
descriptions = [
    "A tale of love and loss set against the backdrop of war.",
    "A gripping mystery where nothing is as it seems.",
    "An epic fantasy adventure in a world of magic and dragons.",
    "A heartwarming story of friendship and second chances.",
    "A chilling thriller that will keep you on the edge of your seat.",
    "A coming-of-age story about finding yourself and your place in the world.",
    "A historical novel that brings the past to life with vivid detail.",
    "A suspenseful crime novel where the detective becomes the hunted.",
    "A dystopian future where one woman's rebellion could change everything.",
    "A romantic comedy that will make you believe in love again.",
    "A science fiction saga that explores the limits of human ingenuity.",
    "A powerful drama about family, secrets, and redemption.",
    "A journey through time to uncover hidden truths.",
    "A modern fairy tale where dreams really do come true.",
    "A dark fantasy filled with intrigue, betrayal, and forbidden magic.",
    "A psychological thriller that will mess with your mind.",
    "A poetic exploration of life, love, and everything in between.",
    "A detective novel where every clue leads to more questions.",
    "A story of survival and the strength of the human spirit.",
    "A heart-pounding adventure in a world beyond our own.",
    "A memoir of a life lived on the edge of society.",
    "A romance that defies the boundaries of time and space.",
    "A political thriller set in a world of corruption and power.",
    "A fantasy epic that weaves together destiny and desire.",
    "A mystery novel where the past refuses to stay buried.",
    "A heartwrenching story of love, loss, and letting go.",
    "A darkly comic tale of life in the absurd.",
    "A science fiction adventure that questions what it means to be human.",
    "A historical romance set in a time of revolution and change.",
    "A supernatural thriller where nightmares come to life.",
    "A journey of self-discovery in a world that demands conformity.",
    "A story of forbidden love in a society bound by tradition.",
    "A fast-paced action novel where every second counts.",
    "A lyrical exploration of nature, solitude, and the passage of time.",
    "A detective story where the truth is stranger than fiction.",
    "A powerful saga of family, loyalty, and betrayal.",
    "A fantastical journey through a land of myths and legends.",
    "A tale of revenge, justice, and the price of power.",
    "A story of hope in the face of overwhelming odds.",
    "A quirky romance where opposites truly attract.",
    "A sci-fi thriller that blurs the line between reality and illusion.",
    "A historical epic that spans generations and continents.",
    "A crime novel where the line between right and wrong is razor-thin.",
    "A love story that unfolds in the most unexpected way.",
    "A philosophical exploration of what it means to live a good life.",
    "A gripping tale of survival in a post-apocalyptic world.",
    "A romance that blossoms in the midst of chaos and war.",
    "A detective novel that unravels the darkest secrets of the human soul.",
    "A story of redemption and the power of forgiveness.",
    "A fantasy adventure where a reluctant hero must save the world."
]

input_text = "A horror novel"

# ...